In [247]:
from __future__ import print_function
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, SpatialDropout1D, TimeDistributed, Reshape, Lambda
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam, SGD
from keras import backend as K
from keras.utils import to_categorical, np_utils
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json, Model
import keras.backend as K
from keras.layers import LSTM, Input
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import pdb
from gensim.models import KeyedVectors
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot

<h3>This step takes forever</h3>

In [8]:
en_model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')

In [87]:
en_model.vocab

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


{',': <gensim.models.keyedvectors.Vocab at 0x2e1852e8>,
 'the': <gensim.models.keyedvectors.Vocab at 0x2e185048>,
 '.': <gensim.models.keyedvectors.Vocab at 0x2e185358>,
 'and': <gensim.models.keyedvectors.Vocab at 0x2e185080>,
 'of': <gensim.models.keyedvectors.Vocab at 0x2e1853c8>,
 'to': <gensim.models.keyedvectors.Vocab at 0x2e1850f0>,
 'in': <gensim.models.keyedvectors.Vocab at 0x2e185438>,
 'a': <gensim.models.keyedvectors.Vocab at 0x2e185160>,
 '"': <gensim.models.keyedvectors.Vocab at 0x2e185470>,
 ':': <gensim.models.keyedvectors.Vocab at 0x2e185198>,
 ')': <gensim.models.keyedvectors.Vocab at 0x2e1854a8>,
 'that': <gensim.models.keyedvectors.Vocab at 0x2e1851d0>,
 '(': <gensim.models.keyedvectors.Vocab at 0x2e185518>,
 'is': <gensim.models.keyedvectors.Vocab at 0x2e185208>,
 'for': <gensim.models.keyedvectors.Vocab at 0x2e185588>,
 'on': <gensim.models.keyedvectors.Vocab at 0x2e185278>,
 '*': <gensim.models.keyedvectors.Vocab at 0x2e1855f8>,
 'with': <gensim.models.keyedvecto

In [11]:
negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                         'even though', 'yet']
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)

In [12]:
def clean(doc):
    lemma=WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop if i not in negative])
    punc_free = "".join([ch for ch in stop_free if ch not in exclude])
    normalized = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized

In [13]:
def vec_word(li):
    total_vecs=[]
    for word in li:
        if word in en_model.vocab:
            vector = en_model[word]
            total_vecs.append(vector)
    return np.array(total_vecs)

In [6]:
len(vec_word('computer'))

300

In [45]:
isear=pd.read_csv('isear_1.csv', header=-1)
isear=isear.drop(2 , axis=1)
isear[0][isear[0]=='guit']='guilt'

In [96]:
X=isear[1]
X=X.apply(clean)
y=isear[0]

In [15]:
def word_splits(series):
    word_splits=series.str.split(' ')
    return word_splits

In [97]:
splits=word_splits(X)
numbers_series=splits.apply(vec_word)

In [113]:
def transform_input(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    num_words=len(numbers_series)
    X_1 = []
    for index in range(0, num_words):
        doc=numbers_series.iloc[index]
        for i in range(3, len(doc)):
            X_1.append(doc[i-3:i])
    return np.array(X_1), len(X_1)

In [59]:
def transform_textform(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits
    X_1 = []
    for index in range(0, len(numbers_series)):
        doc=numbers_series.iloc[index]
        for i in range(3, len(doc)):
            X_1.append(doc[i-3:i])
    return X_1

In [109]:
def transform_textform(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    return splits

In [110]:
transform_textform('I like pie a whole bunch a lot')

0    [like, pie, whole, bunch, lot]
dtype: object

In [213]:
splits

0       [day, feel, close, partner, friend, feel, peac...
1       [every, time, imagine, someone, love, could, c...
2       [obviously, unjustly, treated, possibility, el...
3       [think, short, time, live, relate, period, lif...
4       [gathering, found, involuntarily, sitting, nex...
5       [realized, directing, feeling, discontent, par...
6       [feel, guilty, realize, consider, material, th...
7          [girlfriend, taken, exam, went, parent, place]
8           [when, first, time, realized, meaning, death]
9         [car, overtaking, another, forced, drive, road]
10      [recently, thought, hard, work, take, study, o...
11                   [found, bristle, liver, paste, tube]
12      [tired, unmotivated, shouted, girlfriend, brou...
13      [think, study, enough, weekend, think, able, a...
14                        [pas, examination, think, well]
15      [one, arranged, meet, someone, person, arrives...
16         [one, unjustly, accused, something, one, done]
17      [one, 

In [204]:
len(X)

7516

In [98]:
numbers_series.iloc[1].shape

(11, 300)

<i>Using non-normalized vector numbers? Check</i>

In [46]:
y=isear[0]

In [99]:
X_1 = []
y_1 = []
for index in range(0, len(numbers_series)):
    doc=numbers_series.iloc[index]
    for i in range(3, len(doc)):
        X_1.append(doc[i-3:i])
        y_1.append(y.iloc[index])

In [100]:
type(X_1)

list

In [106]:
X_1[:10]

[array([[ 3.200e-02,  3.810e-02, -2.990e-02, -7.450e-02, -6.240e-02,
         -1.060e-02,  1.214e-01,  6.520e-02, -2.870e-02, -9.300e-03,
          4.320e-02,  1.459e-01, -2.000e-03, -5.200e-02, -5.320e-02,
          2.760e-02,  5.220e-02,  2.200e-03,  1.427e-01,  3.780e-02,
         -1.460e-02, -2.770e-02,  1.090e-02, -3.660e-02, -3.170e-02,
         -3.650e-02, -4.070e-02,  1.970e-02,  5.730e-02, -1.530e-02,
          1.760e-02,  2.430e-02,  7.420e-02, -1.240e-02, -2.500e-02,
         -1.800e-03,  1.190e-02,  1.819e-01, -1.300e-03,  7.220e-02,
         -4.430e-02,  3.330e-02,  2.290e-02, -1.340e-02,  2.340e-02,
         -1.116e-01, -3.580e-02,  1.600e-03, -9.030e-02, -1.252e-01,
         -4.470e-02, -3.710e-02, -6.549e-01,  2.000e-03, -3.950e-02,
         -1.123e-01,  6.580e-02, -6.930e-02, -1.501e-01,  1.480e-02,
         -1.100e-02, -9.800e-02, -6.130e-02,  1.700e-02, -2.390e-02,
         -2.065e-01,  5.090e-02, -6.500e-03, -2.180e-02,  1.390e-02,
         -1.250e-02,  4.410e-02,  

In [103]:
len(X_1)

53460

In [104]:
X_2=np.array(X_1)

In [105]:
X_2.shape

(53460, 3, 300)

In [56]:
encoder=LabelEncoder()
encoder.fit(y_1)
y_1=encoder.transform(y_1)
y=np_utils.to_categorical(y_1)
y

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [231]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y, test_size = 0.2, random_state = 0)
X_train, y_train = np.array(X_train), np.array(y_train)

In [234]:
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [235]:
y_test.shape

(10692, 7)

In [236]:
y_train.shape

(42768, 7)

In [237]:
len(X_train)

42768

In [110]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [111]:
X_train.shape

(53460, 3, 300)

In [112]:
y_train.shape

(53460,)

In [128]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 7))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=50, return_sequences=True, input_shape=(None, 300...)`
  """


In [141]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')
regressor.fit(X_train, y_train, epochs = 25, batch_size = 32)

Epoch 1/25
42768/42768 [==============================] - 18s 419us/step - loss: 0.0495 - acc: 0.7586
Epoch 2/25
42768/42768 [==============================] - 15s 358us/step - loss: 0.0490 - acc: 0.7635
Epoch 3/25
42768/42768 [==============================] - 15s 358us/step - loss: 0.0488 - acc: 0.7621
Epoch 4/25
42768/42768 [==============================] - 15s 359us/step - loss: 0.0488 - acc: 0.7621
Epoch 5/25
42768/42768 [==============================] - 15s 359us/step - loss: 0.0486 - acc: 0.7650
Epoch 6/25
42768/42768 [==============================] - 15s 360us/step - loss: 0.0487 - acc: 0.7630
Epoch 7/25
42768/42768 [==============================] - 15s 362us/step - loss: 0.0485 - acc: 0.7640
Epoch 8/25
42768/42768 [==============================] - 16s 370us/step - loss: 0.0481 - acc: 0.7680
Epoch 9/25
42768/42768 [==============================] - 16s 367us/step - loss: 0.0481 - acc: 0.7662
Epoch 10/25
42768/42768 [==============================] - 16s 384us/step - loss: 

In [142]:
regressor.evaluate(X_test, y_test)

10692/10692 [==============================] - 1s 117us/step


[0.10446360716760003, 0.48110736999625886]

In [135]:
regressor.metrics_names

['loss']

<h3>Initial model with .48 test accuracy</h3>

<h3>Saving and loading</h3>

In [137]:
model_json = regressor.to_json()
with open("emotions_regressor.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
regressor.save_weights("emotions_regressor.h5")
print("Saved model to disk")

Saved model to disk


In [4]:
# load json and create model
json_file = open('emotions_regressor.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("emotions_regressor.h5")
print("Loaded model from disk")

Loaded model from disk


In [58]:
try:
    print(loaded_model.predict(transform_input('I like pie a whole bunch a lot')))
except ValueError:
    print('Input size too small')

[[0.09755597 0.04315098 0.17637494 0.07367538 0.43375415 0.07726857
  0.09988091]
 [0.56483126 0.07285608 0.13253172 0.04942662 0.09893434 0.02065588
  0.0610196 ]]


In [150]:
def predict_avg(text):
    X, X_len=transform_input(text)
    prediction=loaded_model.predict(transform_input(text)[0])
    prediction=np.mean(prediction, axis=0)
    return prediction

In [153]:
pie_mat=predict_avg('I hate all aliens they are evil bastards')
pie_mat

array([0.02197286, 0.90678847, 0.01035543, 0.01425846, 0.00490067,
       0.01707822, 0.01889859], dtype=float32)

0:anger

1:disgust

2:fear

3:guilt

4: joy

5:sadness

6:shame

loaded_model (model 1) doesn't do very well. Let's try to make another

<h3>Model with unknown timespan</h3>

In [203]:
X=isear[1]
X=X.apply(clean)
y=isear[0]

In [204]:
splits=word_splits(X)
numbers_series=splits.apply(vec_word)

In [185]:
def transform_no_window(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    return numbers_series

In [187]:
X=X.apply(transform_no_window)

In [196]:
X_1=X

In [197]:
X_1=X_1[0]

0       [[0.032, 0.0381, -0.0299, -0.0745, -0.0624, -0...
1       [[-0.0036, -0.1675, 0.0635, -0.0249, 0.0098, -...
2       [[0.079, -0.0397, 0.005, 0.0136, -0.0226, -0.0...
3       [[-0.0432, 0.037, -0.0039, 0.0524, -0.1216, -0...
4       [[0.0586, -0.037, -0.081, -0.1985, -0.1255, -0...
5       [[0.1129, -0.0975, 0.0769, -0.071, 0.0645, -0....
6       [[-0.0372, 0.0615, 0.0167, -0.0382, 0.0207, -0...
7       [[0.1912, -0.0745, 0.0554, -0.0277, -0.0276, 0...
8       [[0.0092, -0.0123, -0.0336, 0.0269, 0.031, 0.0...
9       [[-0.016, -0.0003, -0.1684, 0.0899, -0.02, -0....
10      [[0.0103, 0.0984, 0.1326, 0.0845, 0.0898, -0.1...
11      [[-0.0368, -0.0336, 0.0182, 0.0258, -0.0327, -...
12      [[-0.1362, 0.1148, -0.0027, -0.0033, 0.1185, -...
13      [[-0.0432, 0.037, -0.0039, 0.0524, -0.1216, -0...
14      [[-0.0079, -0.0009, -0.0813, -0.2044, 0.1367, ...
15      [[0.0458, -0.0362, 0.0156, 0.0027, -0.053, 0.0...
16      [[0.0458, -0.0362, 0.0156, 0.0027, -0.053, 0.0...
17      [[0.04

In [201]:
X_2=np.array(X_1)
X_2.shape

(7516,)

In [202]:
X_2[0:3]

array([array([[ 0.032 ,  0.0381, -0.0299, ...,  0.1316,  0.1363,  0.044 ],
       [-0.0372,  0.0615,  0.0167, ...,  0.1405, -0.0883, -0.1164],
       [ 0.0622,  0.0628,  0.2393, ...,  0.1802, -0.0394, -0.0243],
       ...,
       [ 0.0104,  0.1101, -0.0982, ...,  0.1598, -0.0329,  0.0798],
       [ 0.0055, -0.1195,  0.0729, ...,  0.2087, -0.0779, -0.0624],
       [-0.1902, -0.1442, -0.1249, ...,  0.2267, -0.0037,  0.0915]],
      dtype=float32),
       array([[-0.0036, -0.1675,  0.0635, ...,  0.1275,  0.1066, -0.0399],
       [ 0.0863, -0.0405,  0.0437, ...,  0.1703, -0.3191, -0.0287],
       [-0.1006, -0.0988, -0.1582, ...,  0.2289,  0.1009, -0.018 ],
       ...,
       [ 0.1084,  0.1093, -0.1632, ..., -0.0037, -0.2236, -0.2337],
       [-0.0112, -0.1144, -0.0171, ...,  0.1761,  0.0193, -0.0004],
       [ 0.1013,  0.0365,  0.0062, ...,  0.0657,  0.0512, -0.0055]],
      dtype=float32),
       array([[ 0.079 , -0.0397,  0.005 , ...,  0.1426, -0.1237, -0.0863],
       [ 0.1821, -0.0348,

In [163]:
encoder=LabelEncoder()
encoder.fit(y)
y=encoder.transform(y)
y=np_utils.to_categorical(y)
y

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, y_train = np.array(X_train), np.array(y_train)

In [171]:
X_train[1].shape

(16, 300)

Input shape

3D tensor with shape (batch_size, timesteps, input_dim).

In [240]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 7))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=50, return_sequences=True, input_shape=(None, 300...)`
  """


In [241]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')

In [ ]:
regressor.fit(new_input, y_train, epochs = 25, batch_size = 32)

In [250]:
I = Input(shape=(None, 300)) # unknown timespan, fixed feature size
lstm = LSTM(20)
dense=Dense(units = 7)
dense_out=dense(I)
f = K.function(inputs=[I], outputs=[lstm(I)])

In [257]:
new_input=list(new_input)

In [259]:
type(new_input)

list

In [ ]:
model=Model(input=[I], output=[dense_out])
model.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')
model.fit(new_input)

In [216]:
X_train[0].shape 

(21, 300)

In [231]:
new_input=list(X_train)
new_input=[np.reshape(arr, (1, arr.shape[0], 300)) for arr in new_input]

In [232]:
f(new_input)

[array([[-0.00189861, -0.02905425,  0.01028993,  0.05693588,  0.12599891,
         -0.07233123, -0.15039034, -0.03753458,  0.0746619 ,  0.03368369,
         -0.01838236, -0.05823456, -0.0047072 , -0.00749667, -0.03643454,
          0.03162642,  0.02647312,  0.02489861,  0.06106438,  0.00751286]],
       dtype=float32)]

In [ ]:
#window
X_1 = []
    for index in range(0, len(numbers_series)):
        doc=numbers_series.iloc[index]
        for i in range(3, len(doc)):
            X_1.append(doc[i-3:i])
    return np.array(X_1)

<h3>Second model</h3>

In [165]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=100, return_sequences=True, input_shape=(None, 300...)`
  """


In [161]:
adam=Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [166]:
# Compiling the RNN
regressor.compile(optimizer = adam, metrics=['accuracy'], loss = 'categorical_crossentropy')

In [173]:
regressor.fit(X_train, y_train, epochs = 25, batch_size = 32)

Epoch 1/25
42768/42768 [==============================] - 32s 748us/step - loss: 0.8835 - acc: 0.6842
Epoch 2/25
42768/42768 [==============================] - 32s 737us/step - loss: 0.8675 - acc: 0.6885
Epoch 3/25
42768/42768 [==============================] - 32s 743us/step - loss: 0.8545 - acc: 0.6947
Epoch 4/25
42768/42768 [==============================] - 31s 735us/step - loss: 0.8418 - acc: 0.7000
Epoch 5/25
42768/42768 [==============================] - 31s 732us/step - loss: 0.8274 - acc: 0.7037
Epoch 6/25
42768/42768 [==============================] - 31s 732us/step - loss: 0.8193 - acc: 0.7093
Epoch 7/25
42768/42768 [==============================] - 32s 750us/step - loss: 0.8059 - acc: 0.71211s - l
Epoch 8/25
42768/42768 [==============================] - 32s 747us/step - loss: 0.8002 - acc: 0.7122
Epoch 9/25
42768/42768 [==============================] - 32s 757us/step - loss: 0.7883 - acc: 0.7179
Epoch 10/25
42768/42768 [==============================] - 33s 765us/step - 

In [174]:
regressor.evaluate(X_test, y_test)

10692/10692 [==============================] - 2s 215us/step


[1.791913282456489, 0.48054620276842497]

In [175]:
model_json = regressor.to_json()
with open("emotions_regressor_2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
regressor.save_weights("emotions_regressor_2.h5")
print("Saved model to disk")

Saved model to disk


<h3>Third model</h3>

In [177]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.5))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100))
regressor.add(Dropout(0.5))

# Adding the output layer
regressor.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=100, return_sequences=True, input_shape=(None, 300...)`
  """


In [180]:
adam=Adam(lr=0.0007, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [181]:
# Compiling the RNN
regressor.compile(optimizer = adam, metrics=['accuracy'], loss = 'categorical_crossentropy')

In [196]:
hist=regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_data=(X_test, y_test), shuffle=False)

Train on 42768 samples, validate on 10692 samples
Epoch 1/100
42768/42768 [==============================] - 33s 783us/step - loss: 0.5579 - acc: 0.8134 - val_loss: 2.3434 - val_acc: 0.4560
Epoch 2/100
 3776/42768 [=>............................] - ETA: 28s - loss: 0.5140 - acc: 0.8284

KeyboardInterrupt: 

In [189]:
regressor.evaluate(X_test, y_test)

10692/10692 [==============================] - 2s 215us/step


[2.214059761745226, 0.4675458286569398]

In [ ]:
pyplot.plot(hist.history['loss'][500:])
pyplot.plot(hist.history['val_loss'][500:])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

<h3>Fourth model</h3>

In [226]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True, input_dim=300))
regressor.add(SpatialDropout1D(0.5))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a LSTM layer
regressor.add(LSTM(units = 100))

# Adding the output layer
regressor.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=100, return_sequences=True, input_shape=(None, 300...)`
  """


In [227]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'categorical_crossentropy')

In [ ]:
X_train.shape

In [238]:
model_4_hist=regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_data=(X_test, y_test), shuffle=False)

Train on 42768 samples, validate on 10692 samples
Epoch 1/100
42768/42768 [==============================] - 43s 1000us/step - loss: 1.8514 - acc: 0.2293 - val_loss: 1.7700 - val_acc: 0.2788
Epoch 2/100
42768/42768 [==============================] - 34s 789us/step - loss: 1.7607 - acc: 0.2812 - val_loss: 1.7198 - val_acc: 0.3098
Epoch 3/100
42768/42768 [==============================] - 34s 789us/step - loss: 1.7203 - acc: 0.3059 - val_loss: 1.6828 - val_acc: 0.3389
Epoch 4/100
42768/42768 [==============================] - 34s 791us/step - loss: 1.6804 - acc: 0.3327 - val_loss: 1.6587 - val_acc: 0.3534
Epoch 5/100
42768/42768 [==============================] - 34s 795us/step - loss: 1.6542 - acc: 0.3474 - val_loss: 1.6329 - val_acc: 0.3690
Epoch 6/100
42768/42768 [==============================] - 34s 789us/step - loss: 1.6295 - acc: 0.3625 - val_loss: 1.6260 - val_acc: 0.3715
Epoch 7/100
42768/42768 [==============================] - 34s 792us/step - loss: 1.6074 - acc: 0.3700 - val_

<h3>Model 5: Stateful learning on size 1 elements</h3>

In [262]:
X=isear[1]
X=X.apply(clean)
y=isear[0]

In [269]:
def transform_train(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    num_words=len(numbers_series)
    X_1 = []
    y_1=  []
    for index in range(0, num_words):
        doc=numbers_series.iloc[index]
        for i in range(len(doc)):
            X_1.append(doc[i])
            y_1.append(y.iloc[index])
    return X_1, y_1

In [290]:
X_1, y_1=transform_train(X)

In [291]:
encoder=LabelEncoder()
encoder.fit(y_1)
y_1=encoder.transform(y_1)
y_1=np_utils.to_categorical(y_1)
y_1

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [293]:
X_1=np.array(X_1)

In [294]:
X_1=np.reshape(X_1, (X_1.shape[0], 1, X_1.shape[1]))

In [296]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size = 0.2, random_state = 0)
X_train, y_train = np.array(X_train), np.array(y_train)

In [312]:
X_train.shape

(58205, 1, 300)

In [322]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25, return_sequences = True, stateful=True,  batch_input_shape=(35, 1, 300)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 7))

In [329]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')
#Fitting to training set
regressor.fit(X_train, y_train, epochs = 25, batch_size = 35)

Epoch 1/25
58205/58205 [==============================] - 11s 187us/step - loss: 0.1149 - acc: 0.2824
Epoch 2/25
58205/58205 [==============================] - 8s 140us/step - loss: 0.1150 - acc: 0.2831
Epoch 3/25
58205/58205 [==============================] - 8s 141us/step - loss: 0.1152 - acc: 0.2803
Epoch 4/25
58205/58205 [==============================] - 8s 140us/step - loss: 0.1150 - acc: 0.2807
Epoch 5/25
58205/58205 [==============================] - 8s 141us/step - loss: 0.1151 - acc: 0.2816
Epoch 6/25
58205/58205 [==============================] - 8s 141us/step - loss: 0.1151 - acc: 0.2788
Epoch 7/25
58205/58205 [==============================] - 8s 142us/step - loss: 0.1153 - acc: 0.2792
Epoch 8/25
58205/58205 [==============================] - 8s 141us/step - loss: 0.1153 - acc: 0.2784
Epoch 9/25
58205/58205 [==============================] - 8s 142us/step - loss: 0.1153 - acc: 0.2774
Epoch 10/25
58205/58205 [==============================] - 8s 143us/step - loss: 0.1154 - 

<h3> Only run below once </h3>

In [325]:
X_test, y_test= X_test[:13475], y_test[:13475]

In [330]:
regressor.evaluate(X_test, y_test, batch_size=35)

13475/13475 [==============================] - 2s 113us/step


[0.11635366178952254, 0.2500185603106564]

In [328]:
regressor.metrics_names

['loss', 'acc']

<i>Notes on size 1: Not as good as size 3

<h3>Model 6: timestep size 6 (double of first model)</h3>

In [331]:
X=isear[1]
X=X.apply(clean)
y=isear[0]

In [421]:
def transform_t6_train(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    num_docs=len(numbers_series)
    X_1 = []
    y_1= []
    for index in range(0, num_docs):
        doc=numbers_series.iloc[index]
        for i in range(6, len(doc)):
            X_1.append(doc[i-6:i])
            y_1.append(y.iloc[index])
    return X_1, y_1

In [533]:
def transform_t6_input(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    num_docs=len(numbers_series)
    for index, doc in enumerate(numbers_series):
        while len(doc)<6:
            orig_doc=doc.copy()
            orig_doc=list(orig_doc)
            doc=list(doc)
            for word in orig_doc:
                doc.append(word)
                #doc=np.insert(doc,(len(doc)),word, axis=0)
                #doc=np.append(doc, word, axis=1)
            modified=True
        doc=np.array(doc)
    X_1 = []
    print(numbers_series.iloc[0])
    if num_docs>1:
        for index in range(0, num_docs):
            doc=numbers_series.iloc[index]
            for i in range(6, len(doc)):
                X_1.append(doc[i-6:i])
    else:
        doc=numbers_series.iloc[0]
        print(doc.shape)
        for i in range(6, len(doc)):
                X_1.append(doc[i-6:i])
    return X_1

In [515]:
def test(text):
    text=('I like pie')
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    doc=numbers_series.iloc[0]
    return doc

In [522]:
pie=list(test('I like pie'))

In [523]:
pie[0].shape

(300,)

In [528]:
pie.append(pie[0])

In [529]:
pie

[array([-6.370e-02, -7.300e-02, -4.340e-02,  4.290e-02, -6.410e-02,
        -1.114e-01,  2.320e-02, -3.760e-02,  7.790e-02,  1.435e-01,
        -4.800e-03, -6.900e-03,  2.220e-02, -4.310e-02, -1.970e-02,
         8.110e-02,  2.380e-02,  1.243e-01, -6.790e-02,  9.600e-03,
        -1.630e-02,  5.360e-02,  2.680e-02,  4.240e-02, -7.510e-02,
        -8.090e-02, -5.100e-02,  2.600e-02, -6.310e-02, -2.150e-02,
         4.990e-02, -6.130e-02, -1.650e-02, -4.610e-02,  2.860e-02,
         6.960e-02, -5.380e-02,  2.151e-01, -3.750e-02, -2.090e-02,
        -1.380e-02, -4.200e-03,  9.200e-03,  1.620e-02, -4.450e-02,
        -2.250e-02, -3.620e-02,  5.680e-02,  3.640e-02,  1.200e-02,
        -1.026e-01, -4.522e-01, -6.724e-01,  2.400e-03,  7.560e-02,
        -2.350e-02, -3.180e-02,  4.270e-02, -2.900e-03, -8.110e-02,
        -4.490e-02, -8.720e-02,  7.950e-02,  9.100e-03,  5.670e-02,
        -8.190e-02,  4.800e-02,  4.370e-02,  6.250e-02, -3.770e-02,
         2.920e-02,  3.980e-02, -2.680e-02, -1.2

In [534]:
transform_t6_input('I like pie')

[[-6.370e-02 -7.300e-02 -4.340e-02  4.290e-02 -6.410e-02 -1.114e-01
   2.320e-02 -3.760e-02  7.790e-02  1.435e-01 -4.800e-03 -6.900e-03
   2.220e-02 -4.310e-02 -1.970e-02  8.110e-02  2.380e-02  1.243e-01
  -6.790e-02  9.600e-03 -1.630e-02  5.360e-02  2.680e-02  4.240e-02
  -7.510e-02 -8.090e-02 -5.100e-02  2.600e-02 -6.310e-02 -2.150e-02
   4.990e-02 -6.130e-02 -1.650e-02 -4.610e-02  2.860e-02  6.960e-02
  -5.380e-02  2.151e-01 -3.750e-02 -2.090e-02 -1.380e-02 -4.200e-03
   9.200e-03  1.620e-02 -4.450e-02 -2.250e-02 -3.620e-02  5.680e-02
   3.640e-02  1.200e-02 -1.026e-01 -4.522e-01 -6.724e-01  2.400e-03
   7.560e-02 -2.350e-02 -3.180e-02  4.270e-02 -2.900e-03 -8.110e-02
  -4.490e-02 -8.720e-02  7.950e-02  9.100e-03  5.670e-02 -8.190e-02
   4.800e-02  4.370e-02  6.250e-02 -3.770e-02  2.920e-02  3.980e-02
  -2.680e-02 -1.215e-01  2.080e-02  3.080e-02  2.720e-02  1.110e-02
   7.430e-02  1.202e-01  6.020e-02 -7.000e-03 -2.600e-02 -3.549e-01
  -8.000e-04  7.620e-02  5.140e-02 -1.210e-02 -1

[]

In [333]:
X_1, y_1=transform_t6_train(X)

In [336]:
X_1=np.array(X_1)
X_1.shape

(32468, 6, 300)

In [335]:
encoder=LabelEncoder()
encoder.fit(y_1)
y_1=encoder.transform(y_1)
y_1=np_utils.to_categorical(y_1)
y_1

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [337]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size = 0.2, random_state = 0)
X_train, y_train = np.array(X_train), np.array(y_train)

In [339]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 7))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=25, return_sequences=True, input_shape=(None, 300...)`
  """


In [351]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')
#Fitting to training set
regressor.fit(X_train, y_train, epochs = 25)

Epoch 1/25
25974/25974 [==============================] - 17s 637us/step - loss: 0.0240 - acc: 0.9157
Epoch 2/25
25974/25974 [==============================] - 12s 470us/step - loss: 0.0236 - acc: 0.9177
Epoch 3/25
25974/25974 [==============================] - 12s 462us/step - loss: 0.0239 - acc: 0.9175
Epoch 4/25
25974/25974 [==============================] - 12s 470us/step - loss: 0.0238 - acc: 0.9171
Epoch 5/25
25974/25974 [==============================] - 12s 468us/step - loss: 0.0233 - acc: 0.9191
Epoch 6/25
25974/25974 [==============================] - 12s 467us/step - loss: 0.0237 - acc: 0.9173
Epoch 7/25
25974/25974 [==============================] - 12s 472us/step - loss: 0.0232 - acc: 0.9202
Epoch 8/25
25974/25974 [==============================] - 12s 468us/step - loss: 0.0232 - acc: 0.9205
Epoch 9/25
25974/25974 [==============================] - 12s 473us/step - loss: 0.0232 - acc: 0.9193
Epoch 10/25
25974/25974 [==============================] - 12s 469us/step - loss: 

In [352]:
regressor.evaluate(X_test, y_test)

6494/6494 [==============================] - 2s 356us/step


[0.05276456107346735, 0.7685555896466792]

In [375]:
def prediction(text):
    vec_text=transform_t6_input(text)
    if len(vec_text)<6:
        vec_B=vec_text.copy()
        while len(vec_text)<6:
            vec_text.append('a')
            for vec in vec_B:
                print(vec)
                ##if len(vec_text)<6:
                #    vec_text.append(vec)
    #y=regressor.predict(vec_text)
    return vec_text

In [440]:
X_99=transform_t6_input('I like pie')
X_99=np.array(X_99)
X_99.shape

1200

In [356]:
def prediction(text):
    vec_text=transform_t6_input(text)
    return type(vec_text)

In [376]:
prediction('I like to eat pie a lot it is yummy and very delicious')

['a', 'a', 'a', 'a', 'a', 'a']

<h3>Further options</h3>

I could 

1.Gather and combine many emotional labelled datasets to have larger training data

2.Create new training data synthetically

3.Create labelled data using word averages (auto-labelling) data

4.Use Indico to get around 10,000 more labelled training data rows

<strong>Idea:</strong>Duplicate input if length of input too short

<strong>Indico's emotions</strong>:
Anger, Fear, Joy, Sadness, Surprise

<strong>ISEAR's emotions</strong>: Anger, Fear, Joy, Sadness, Disgust, Shame, Guilt


<h1>Input shape

3D tensor with shape (batch_size, timesteps, input_dim)</h1>

0:anger

1:disgust

2:fear

3:guilt

4: joy

5:sadness

6:shame